<a href="https://colab.research.google.com/github/XinzhouLi/Toxic_Language_Detection_in_Social_Media/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! git clone https://github.com/XinzhouLi/Toxic_Language_Detection_in_Social_Media.git
! pip install torch==1.8 torchtext==0.9

fatal: destination path 'Toxic_Language_Detection_in_Social_Media' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu

 https://www.cnblogs.com/cxq1126/p/13466998.html
 这篇文章 有讲述如何怎么建立完整的训练过程


 https://www.cnblogs.com/cxq1126/p/13504437.html
 这个就是我们的最终目标 LSTM加上Attention


 https://www.cnblogs.com/cxq1126/p/13508696.html
 这个是transformer的文本分类我还没看先扔这


我相信都接触过colab吧， 因为需要大量运算基本都不在本地跑，全都白嫖Google的算力， 
然后上面那一行是先把文件克隆到自己的drive里面这样就省的自己上传文件了
然后你们可以先看看， 我基本都写了comment， 有一些没写的十有八九我也是抄的
所以我也不一定知道那些代码是干嘛的， 也试试我的那些variable里面都存了啥， 每一行改一改试试，看看都是干嘛的。等之后见面开会再一起写写


In [4]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
from torch import nn, optim
import torchtext
import re
import sys

In [5]:
print(torch.__version__)
print(sys.version)
print(torchtext.__version__)

1.8.0
3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
0.9.0


In [6]:
def unify_format(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text).lower().split()

In [7]:
# implement dataset extends util.data.Dataset
class ToxicDataset(data.Dataset):
    def __init__(self, *args):
        super().__init__()
        # Read data in
        if len(args) == 1:
            # for initialize training dataset
            filepath = args[0]
            self.dataframe = pd.read_csv(
                filepath, iterator=True, header=0, encoding='utf-8', usecols=['comment_text', 'toxic'])
            # initialize pandas dataframe to store the data
            self.dataframe = pd.DataFrame(self.dataframe.read())
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(
                unify_format)
        else:
            # for initialize testing dataset
            filepath1 = args[0]
            filepath2 = args[1]
            test_comments = pd.DataFrame(pd.read_csv(
                filepath1, iterator=True, header=0, usecols=['comment_text', 'id']).read())
            test_label = pd.DataFrame(pd.read_csv(
                filepath2, iterator=True, header=0, usecols=['id', 'toxic']).read())
            # merge two dataframe together by unique key id
            result = pd.merge(test_comments, test_label, how='left', on=['id'])
            # select the text and label col
            self.dataframe = result[result['toxic']>= 0].loc[:, ['comment_text', 'toxic']]
            self.dataframe.convert_dtypes()
            # Clean the data using regular expersion, only reserve letter and number
            self.dataframe['comment_text'] = self.dataframe['comment_text'].apply(unify_format)

    # override the getiem function, return tuple contains comment and label

    def __getitem__(self, index):
        comment, label = self.dataframe.iat[index, 0], self.dataframe.iat[index, 1]
        return comment, label

    # Override the len function, return the length of the dataframe
    def __len__(self):
        return len(self.dataframe.loc[:, ['comment_text']])


In [8]:
# init dataset
# train_dataset = ToxicDataset("train.csv")
# test_dataset = ToxicDataset("test.csv", "test_labels.csv")
train_dataset = ToxicDataset("/content/Toxic_Language_Detection_in_Social_Media/train.csv")
test_dataset = ToxicDataset("/content/Toxic_Language_Detection_in_Social_Media/test.csv", "/content/Toxic_Language_Detection_in_Social_Media/test_labels.csv")
# init dataloader
train_dataloader = data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=100, shuffle=True)


In [15]:
TEXT = torchtext.legacy.data.Field(tokenize='spacy')

/usr/local/lib/python3.8/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [16]:
class Network(nn.Module):
    def __init__(self, vocab_size, embeding_dim, hidden_dim):
        super().__init__()
        # implement embedding function to convert text to numerical data
        self.embedding = nn.Embedding(vocab_size, embeding_dim)
        self.rnn = nn.LSTM(embeding_dim, hidden_dim, num_layers =2, bidirectional=True, dropout = 0.5)
        self.fc = nn.Linear(hidden_dim*2,1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        embediding = self.dropout(self.embedding(x))
        output,(hidden, cell) = self.rnn(embediding)
        hidden = torch.cat([hidden[-2], hidden[-1]],dim=1)
        hidden = self.dropout(hidden)
        out = self.fc(hidden)
        return out


In [10]:
pretrained = torchtext.vocab.GloVe()
# 这块我也不知道怎么处理， 但本质上就是里面有个nn.embedding layer， 应该用别人已经算好的Glove来替代nn中本身自带的， 但没办法替换不了

.vector_cache/glove.840B.300d.zip: 2.18GB [08:24, 4.31MB/s]                            
100%|█████████▉| 2196016/2196017 [03:36<00:00, 10131.72it/s]


In [12]:
embeding_size = 300
vocab_size = 10000
# init Lstm model 
BLSTM = Network(vocab_size, embeding_size, 256)
# replace the embedding to the pretrained embedding Glove
BLSTM.embedding = torch.nn.Embedding.from_pretrained(pretrained.vectors,freeze=True) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(BLSTM.parameters(), lr=1e-3)
criteon = nn.BCEWithLogitsLoss().to(device)
BLSTM.to(device)

Network(
  (embedding): Embedding(2196017, 300)
  (rnn): LSTM(300, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [15]:
# training process
# 没写完
def binary_acc(preds, y):
    """
    get accuracy
    """
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(rnn, iterator, optimizer, criteon):
    
    avg_acc = []
    rnn.train()
    
    for i, batch in enumerate(iterator):
        
        # [seq, b] => [b, 1] => [b]
        pred = rnn(batch.text).squeeze(1)
        # 
        loss = criteon(pred, batch.label)
        acc = binary_acc(pred, batch.label).item()
        avg_acc.append(acc)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print(i, acc)
        
    avg_acc = np.array(avg_acc).mean()
    print('avg acc:', avg_acc)		

train(BLSTM, train_dataloader, optimizer, criteon)


RuntimeError: ignored